<a href="https://colab.research.google.com/github/Gaurav7004/RSE_TOOL/blob/main/RSE_TOOL_12thApril2023_2_20_PM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install pandarallel
# from pandarallel import pandarallel
import json
import re
import pandas as pd
import numpy as np
import itertools

In [2]:
# pandarallel.initialize()

In [3]:
df = pd.read_csv('Data1.csv')

# remove blank spaces from front and back of column names
df.columns = df.columns.str.strip()
df.columns

<ipython-input-3-fc8b3e05b5c4>:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Data1.csv')


Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [4]:
## Data is the dictionary
data = {}

## Keys
key = ''

## values
values = []

## Open
with open('Example2.RSE', "r") as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            if key:
                key = key.split("'")[0]
                key = key.split("\t")[0]
                data[key] = [v for v in values if v and not v.startswith("'")]
            key = line.split("'")[0]
            key = key.split("\t")[0]
            values = []
        elif line.startswith("'"):
            pass
        else:
            line = line.split("'")[0]
            values.append(line.strip())

# add last group
if key:
    data[key] = [v for v in values if v and not v.startswith("'")]

print(json.dumps(data, indent=4))

{
    "#SAMPLING SUBSAMPLE2": [],
    "#GROUP 2": [
        "SEC   C sec",
        "AGE_GR C NEWVAR"
    ],
    "#FILTER 1": [
        "AGE>=15"
    ],
    "#VARIABLE 3  ": [
        "POP\tN  NEWVAR",
        "LF\tN  NEWVAR",
        "WRK\tN  NEWVAR"
    ],
    "#FILE Data1.Xlsx": [],
    "#RENAME  5    ": [
        "STRMID C sec+nss+strm+sstrm",
        "SS     C ss",
        "NSS    N  nss",
        "NSC    N  nsc",
        "MULT   N  mult"
    ],
    "#TRANSFORM 4  ": [
        "AGE_GR=(\"01\") AGE<15",
        "AGE_GR=(\"02\") AGE>=15 && AGE<20",
        "AGE_GR=(\"03\") AGE>=20 && AGE<30",
        "AGE_GR=(\"04\") AGE>=30 && AGE<60",
        "AGE_GR=(\"05\") Otherwise",
        "POP=(1)",
        "WRK=(1)  CWS in (\"11,12,21,31,41,51,61,61,71,72\")",
        "WRK=(0)  CWS NOT in (\"11,12,21,31,41,51,61,61,71,72\")",
        "LF=(1)   CWS in (\"11,12,21,31,41,51,61,61,71,72,81\")",
        "LF=(0)   Otherwise"
    ],
    "#EST_RSE 7": [
        "S POP",
        "E POP",
        "R 

In [5]:
def SAMPLING_(data, df):
    for key, value in data.items():
        ## To read the sampling method used
        if str('SAMPLING').lower() in str(key).lower():
            if str('SUBSAMPLE2').lower() in str(key).lower():
                return str('SUBSAMPLE2')

            elif str('SRSWR').lower() in str(key).lower():
                pass
            elif str('SRSWOR').lower() in str(key).lower():
                pass

SAMPLING_(data, df)

'SUBSAMPLE2'

In [6]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG'],
      dtype='object')

In [7]:
def GROUP_(data, df):
    for key, value in data.items():    
        if str('GROUP').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            for name in value:
                if name in df.columns:
                    pass
                elif name not in df.columns:
                    df[name] = 0

            return value


    for key, value in data.items():
        ### To read the group columns
        if str('GROUP').lower() in str(key).lower():
            # Define regular expression to match three words separated by whitespace
            regex = r'\w+\s+\w+\s+\w+'

            for i in range(len(value)):
                # Find the first match in the string
                match = re.search(regex, value[i])

                # Print the match
                lst = match.group().split()

                if lst[0] in df.columns:
                    if lst[1] == 'C':
                        df[lst[0]] = df[lst[0]].astype(str)
                elif lst[0] not in df.columns:
                    # if lst[1] == 'C':
                    #     df[lst[0]] = ''
                    pass

        else:
            pass

def FILTER_(data, df):
    for key, value in data.items():
        ### To read the sampling method used
        if str('FILTER').lower() in str(key).lower():
            ### Replace logical AND '&&' with 'and'
            if '&&' in value[0]:
                return value[0].replace('&&', '&')
            ### Replace logical OR '||' with 'or'
            elif '||' in value[0]:
                return value[0].replace('||', '|')
            else:
                return value[0]

def FILE_(data):
    for key, value in data.items():
        ### To read the file used
        if str('FILE').lower() in str(key).lower():
            if key.strip().endswith('.xlsx') or key.strip().endswith('.csv'):
                match = re.search(r'[A-Za-z0-9]+\.xlsx|[A-Za-z0-9]+\.csv', key)   
                if match:
                    file_name = match.group(0)
                    return file_name
                else:
                    file_name = None
                    return file_name
                print(file_name)

            # return file_name

def RENAME_(data, df):
    # create an empty dictionary to store the column names
    column_dict = {}

    for key, value in data.items():
        ### To read the file used
        if str('RENAME').lower() in str(key).lower():
            # value = [elem for elem in value[0].split(' ') if elem.strip()]
            # return value

            for col in value:
                col_split = col.split()

                # print(col_split[0])

                # extract the last element as the column name
                col_name = col_split[-1]

                new_col_name = col_split[0]

                ### if there is a "+" sign in the last element, concatenate the columns
                if "+" in col_name:
                    col_concat = col_name.split("+")
                    col_concat = [i.upper() for i in col_concat]

                    # concatenate the columns
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    # df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if i < 10 else str(i) for i in x]), axis=1)
                    df[new_col_name] = df.loc[:, col_concat].apply(lambda x: ''.join(['0' + str(i) if int(i) < 10 else str(i) for i in x]), axis=1)


                else:
                    col_name = col_name.upper()
                    print(type(col_name))


def NEWVARIABLE_(data, df):
    for key, value in data.items():
        ### To read the file used
        if str('VARIABLE').lower() in str(key).lower():
            value = [s.split()[0] for s in value]
            
            for name in value:
                df[name] = 1


def TRANSFORM_(data):
    global condition

    # Initialize empty arrays to store the values
    Arr1 = []
    Arr2 = []
    Arr3 = []

    for key, value in data.items():
        ### To read the file used
        if str('TRANSFORM').lower() in str(key).lower():
            lines = '\n'.join(value)
            lines = lines.split("\n")

            # Loop through the lines and extract the values
            for line in lines:
                line_values = line.split('=')  # split the line into values
                print(line_values)
                var_name = line_values[0].strip()  # extract the variable name
                
                var_value = line_values[1].strip().split()[0]  # extract the variable value

                            
                # Regular expression pattern to extract the condition
                pattern = r'^\w+=\([^\)]*\)(?:\s+(.*?)(?:\n|$))?$'

                # Split the string into lines and extract the condition from each line
                conditions = []

                for line in line.split('\n'):
                    match = re.match(pattern, line)
                 
                    if match:
                        condition = match.group(1)
                        
                        if condition is not None:
                            conditions.append(condition.strip())
                        else:
                            conditions.append(None)

                # Append the values to the arrays
                Arr1.append(var_name)
                Arr2.append(var_value)

                Arr3.append(condition)

    return Arr1, Arr2, Arr3
        

def EST_RSE_(data):
    for key, value in data.items():
        pass

GROUP_(data, df)
TRANSFORM_(data)
RENAME_(data, df)
NEWVARIABLE_(data, df)
FILE_(data)
FILTER_(data, df)


['AGE_GR', '("01") AGE<15']
['AGE_GR', '("02") AGE>', '15 && AGE<20']
['AGE_GR', '("03") AGE>', '20 && AGE<30']
['AGE_GR', '("04") AGE>', '30 && AGE<60']
['AGE_GR', '("05") Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)   CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)   Otherwise']
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


'AGE>=15'

In [8]:
df['SEC'].value_counts()

1    62248
2    47608
Name: SEC, dtype: int64

In [9]:
def parse_query_string(query_string):
    # Replace curly quotes with straight quotes
    query_string = query_string.replace("”", "\"")
    
    if 'in' in query_string:
        if ('NOT in' or 'not in') in query_string:
            query_string = re.sub(r'(\b\w+\b)\s+NOT in\s*\(([^)]+)\)', r'~\1.isin([\2])', query_string)
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        elif 'in' in query_string:
            # Replace 'in' operators with 'isin' and add list brackets
            query_string = re.sub(r'(\b\w+\b)\s+in\s*\(([^)]+)\)', r'\1.isin([\2])', query_string)
        else:
            pass
            
    else:
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)('[^']*'|"[^"]*")|(\b\w+\b)(\s*[<>=!]+\s*)("[^]*'|"[^"]*")''', r'\1\2\3', query_string)
        query_string = re.sub(r'''(\b\w+\b)(\s*[<>=!]+\s*)(\d+)''', r'\1\2\3', query_string)

    # Replace double quotes with single quotes
    query_string = query_string.replace("\"", "'")
    
    # Replace 'AND' with '&', 'OR' with '|', and add parentheses
    query_string = query_string.replace(" AND ", " & ")\
                        .replace(" OR ", " | ").\
                            replace("&&", '&').\
                                replace('||', '|').\
                                    replace('<>', '!=')
    
    # Replace column names with df[column] syntax
    query_string = re.sub(r'(\b\w+\b)=', r'df["\1"]==', query_string)


    # Define a function to replace variable names with df["<variable_name>"]
    def replace_var(match):
        return f'df["{match.group(0)}"]'

    # Use regex to replace variable names with df["<variable_name>"]
    pattern = re.compile(r'''\b\w+\b(?=\s*[<>=!]=*\s*[\'"\d])''')
    new_string = pattern.sub(replace_var, query_string)

    query_string = re.sub(r"'(\d+)'|\"(\d+)\"", lambda match: str(int(match.group(1) or match.group(2))), query_string)
    query_string = re.sub(r'\[(\d+,\s*)+\d+\]', lambda match: '[' + match.group(0).replace("\'", "'") + ']', query_string)
    
    return query_string


final_parsed_query = []

for i in list(TRANSFORM_(data))[2]:
    # print(i)
    if i:
        parsed_query = parse_query_string(i)
        final_parsed_query.append(parsed_query)
    else:
        final_parsed_query.append(i)

# new_lst = [s.replace("['", "[").replace("']", "]") for s in final_parsed_query]
# print([s.replace("['", "[").replace("']", "]") for s in final_parsed_query if s else pass])
# final_parsed_query = [s.replace("['", "[").replace("']", "]") if s is not None else None for s in final_parsed_query]
final_parsed_query

['AGE_GR', '("01") AGE<15']
['AGE_GR', '("02") AGE>', '15 && AGE<20']
['AGE_GR', '("03") AGE>', '20 && AGE<30']
['AGE_GR', '("04") AGE>', '30 && AGE<60']
['AGE_GR', '("05") Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)   CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)   Otherwise']


['AGE<15',
 'AGE>=15 & AGE<20',
 'AGE>=20 & AGE<30',
 'AGE>=30 & AGE<60',
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [10]:
df['SEC']

0         1
1         1
2         1
3         1
4         1
         ..
109851    2
109852    2
109853    2
109854    2
109855    2
Name: SEC, Length: 109856, dtype: int64

In [11]:
Arr1 = list(TRANSFORM_(data))[0]
Arr2 = list(TRANSFORM_(data))[1]

# print(Arr2)
Arr2 = [i.strip('()\'\"') for i in Arr2]
Arr3 = final_parsed_query

Arr2 = [float(elem) if elem.replace('.', '', 1).isdigit() else elem for elem in Arr2]
print(Arr2)

# print(Arr1)
# print(Arr2)
# print(Arr3)

['AGE_GR', '("01") AGE<15']
['AGE_GR', '("02") AGE>', '15 && AGE<20']
['AGE_GR', '("03") AGE>', '20 && AGE<30']
['AGE_GR', '("04") AGE>', '30 && AGE<60']
['AGE_GR', '("05") Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)   CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)   Otherwise']
['AGE_GR', '("01") AGE<15']
['AGE_GR', '("02") AGE>', '15 && AGE<20']
['AGE_GR', '("03") AGE>', '20 && AGE<30']
['AGE_GR', '("04") AGE>', '30 && AGE<60']
['AGE_GR', '("05") Otherwise']
['POP', '(1)']
['WRK', '(1)  CWS in ("11,12,21,31,41,51,61,61,71,72")']
['WRK', '(0)  CWS NOT in ("11,12,21,31,41,51,61,61,71,72")']
['LF', '(1)   CWS in ("11,12,21,31,41,51,61,61,71,72,81")']
['LF', '(0)   Otherwise']
[1.0, 2.0, 3.0, 4.0, 5.0, 1.0, 1.0, 0.0, 1.0, 0.0]


In [12]:
df.head(3)

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,PAS,PS_SS,CWS,ERN_SELF,ERN_REG,AGE_GR,STRMID,POP,LF,WRK
0,1,28,17469,1,1,1,1,1,1,1,...,51,51,51,0,0,0,01010101,1,1,1
1,1,28,17469,1,1,1,1,1,1,1,...,91,91,91,0,0,0,01010101,1,1,1
2,1,28,17469,1,1,1,1,1,1,1,...,91,91,91,0,0,0,01010101,1,1,1


In [13]:
set(Arr1) - set(list(df.columns))

set()

In [14]:

# Check for name mismatches and create new columns
mismatched_names = set(Arr1) - set(df.columns)

new_columns = {name: [0] * df.shape[0] for name in mismatched_names}
df = df.assign(**new_columns)

mismatched_names

set()

In [15]:
Arr3

['AGE<15',
 'AGE>=15 & AGE<20',
 'AGE>=20 & AGE<30',
 'AGE>=30 & AGE<60',
 'Otherwise',
 None,
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "~CWS.isin(['11,12,21,31,41,51,61,61,71,72'])",
 "CWS.isin(['11,12,21,31,41,51,61,61,71,72,81'])",
 'Otherwise']

In [16]:
Arr3 = [i.replace("(['", "([").replace("'])", "])") if i is not None else 'True' if Arr3[j] == 'Otherwise' else True for j, i in enumerate(Arr3)]
Arr3

['AGE<15',
 'AGE>=15 & AGE<20',
 'AGE>=20 & AGE<30',
 'AGE>=30 & AGE<60',
 'Otherwise',
 True,
 'CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 'CWS.isin([11,12,21,31,41,51,61,61,71,72,81])',
 'Otherwise']

In [17]:
new_arr3 = []
for i, value in enumerate(Arr3):
    if value == 'Otherwise':
        other_values = set([Arr3[j] for j in range(len(Arr3)) if Arr1[j] == Arr1[i] and Arr3[j] != 'Otherwise'])
        new_value = '~' + ' & ~'.join(other_values)
        new_arr3.append(new_value)
    else:
        new_arr3.append(value)

new_arr3

['AGE<15',
 'AGE>=15 & AGE<20',
 'AGE>=20 & AGE<30',
 'AGE>=30 & AGE<60',
 '~AGE>=20 & AGE<30 & ~AGE>=15 & AGE<20 & ~AGE<15 & ~AGE>=30 & AGE<60',
 True,
 'CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72])',
 'CWS.isin([11,12,21,31,41,51,61,61,71,72,81])',
 '~CWS.isin([11,12,21,31,41,51,61,61,71,72,81])']

In [18]:
Arr3 = new_arr3

In [19]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'AGE_GR',
       'STRMID', 'POP', 'LF', 'WRK'],
      dtype='object')

In [20]:
Arr2

[1.0, 2.0, 3.0, 4.0, 5.0, 1.0, 1.0, 0.0, 1.0, 0.0]

In [21]:
for i in range(len(Arr2)):
    if type(Arr2[i]) == str:
        print(Arr2[i])

In [22]:
# Apply the queries in Arr3 to filter the rows of the dataframe
for i, query_str in enumerate(Arr3):
    
    if query_str is True:
        if type(Arr2[i]) == str:
            if '+' in Arr2[i]:
                df[Arr1[i]] = df[Arr2[i].split('+')].apply(lambda x: x.sum(), axis=1)
                print('*** First if', Arr2[i], Arr1[i], Arr3[i], i)

            else:
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                print('*** First else', Arr2[i], Arr1[i], Arr3[i], i)
        # if type(Arr3[i]):
        #     print('^^^ Second if', Arr2[i], Arr1[i], Arr3[i], i)
        #     df.loc[df_query.index, Arr1[i]] = Arr2[i]
    
    else:
        try:
            df_query = df.query(query_str)
        except:
            pass

        if type(Arr2[i]) == float:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('### Third if', Arr2[i], Arr1[i], Arr3[i])
        else:
            df.loc[df_query.index, Arr1[i]] = Arr2[i]
            # print('@@@ First else', Arr2[i], Arr1[i], Arr3[i])


        if type(Arr2[i]) == str:
            if Arr2[i] in list(df.columns):
                df[Arr1[i]] = df[[Arr2[i]]].apply(lambda x: x, axis=1)
                # print('$$$ Fourth if', Arr2[i], Arr1[i], Arr3[i])

# print(df)

In [23]:
df[['POP']].value_counts()

POP
1      109856
dtype: int64

In [24]:
df

,SEC,ST,FSU,IDNO,SEG,SSS,SSU,STRM,SSTRM,SS,...,PAS,PS_SS,CWS,ERN_SELF,ERN_REG,AGE_GR,STRMID,POP,LF,WRK
0,1,28,17469,1,1,1,1,1,1,1,...,51,51,51,0,0,5,01010101,1,1,1
1,1,28,17469,1,1,1,1,1,1,1,...,91,91,91,0,0,2,01010101,1,0,0
2,1,28,17469,1,1,1,1,1,1,1,...,91,91,91,0,0,2,01010101,1,0,0
3,1,28,17469,1,1,1,1,1,1,1,...,51,51,51,0,0,5,01010101,1,1,1
4,1,28,17469,2,1,1,2,1,1,1,...,51,51,51,0,0,3,01010101,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109851,2,34,27428,7,1,4,1,1,3,2,...,94,94,94,0,0,0,02030103,1,0,0
109852,2,34,27428,8,1,4,2,1,3,2,...,51,51,51,0,0,5,02030103,1,1,1
109853,2,34,27428,8,1,4,2,1,3,2,...,91,91,91,0,0,1,02030103,1,0,0
109854,2,34,27428,8,1,4,2,1,3,2,...,92,92,92,0,0,5,02030103,1,0,0


In [25]:
df.to_csv('WorkFile_Balanced_Half_Sample_Method_Example5.csv')

In [26]:
# # Example dataframe
# df = pd.DataFrame({
#     'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
#     'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
#     'C': [1, 2, 3, 4, 5, 6, 7, 8],
#     'D': [10, 20, 30, 40, 50, 60, 70, 80],
#     'E': ['foo1', 'bar1', 'foo1', 'bar1', 'foo1', 'bar1', 'foo1', 'foo1'],
#     'F': ['one2', 'one2', 'two2', 'three2', 'two2', 'two2', 'one2', 'three2'],
#     'G': [101, 201, 301, 401, 501, 601, 701, 801],
# })

# # Define the prefix to use for the new columns
# prefix = 'Z'

# lst = ['T1', 'T2', 'T3']

# for idx, col in enumerate(df.columns):
#     for i in range(len(lst)):
#         # Create a new column with the appropriate number of "Z"s and the prefix
#         new_col_val = prefix * (i+1)
#         # Add the new column to the DataFrame
#         df[lst[i]] = new_col_val


# # Print the resulting DataFrame
# print(df)

In [27]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'AGE_GR',
       'STRMID', 'POP', 'LF', 'WRK'],
      dtype='object')

In [28]:
# import pandas as pd
# import itertools

# # Example dataframe
# df = pd.DataFrame({
#     'A': ['foo', 'bar', 'foo', 'bar', 'foo', 'bar', 'foo', 'foo'],
#     'B': ['one', 'one', 'two', 'three', 'two', 'two', 'one', 'three'],
#     'C': [1, 2, 3, 4, 5, 6, 7, 8],
#     'D': [10, 20, 30, 40, 50, 60, 70, 80],
#     'E': ['foo1', 'bar1', 'foo1', 'bar1', 'foo1', 'bar1', 'foo1', 'foo1'],
#     'F': ['one2', 'one2', 'two2', 'three2', 'two2', 'two2', 'one2', 'three2'],
#     'G': [101, 201, 301, 401, 501, 601, 701, 801],
# })

# lst_df = list(df.columns)

# # Select columns to group by
# groupby_cols = ['A', 'B', 'E', 'C']

# # Create a list of all possible subsets of groupby_cols
# groupby_cols_subsets = [list(subset) for i in range(len(groupby_cols) + 1)
#                         for subset in itertools.combinations(groupby_cols, i)]

# print(groupby_cols_subsets)

# # Select columns that are not in groupby_cols
# cols = [col for col in df.columns if col not in groupby_cols]

# agg_dict = {col: sum for col in cols if df[col].dtype == int or df[col].dtype == float}

# i = 0
# # Loop over each subset of columns and perform the aggregation
# for groupby_cols_subset in groupby_cols_subsets:
#     # Define the prefix to use for the new columns
#     prefix = 'Z'

#     if len(groupby_cols_subset) == 0:

#         # # If empty list is selected, perform aggregation without grouping
#         new_df = df.agg(agg_dict).to_frame().T

#         test_lst = list(set(lst_df) - set(list(new_df.columns)))

#         new_df['groupby_cols'] = 'All columns'

#         new_col_val_ = []
#         for i in range(len(test_lst)):
#             # Create a new column with the appropriate number of "Z"s and the prefix
#             new_col_val_.append(prefix * (i+1))

#         for i in range(len(new_col_val_)):
#             # Add the new column to the DataFrame
#             new_df[test_lst[i]] = new_col_val_[i]
        
#         i += 1
#         print(new_df, '\n')

#         continue

#     # Create a new dataframe by grouping by the selected columns and aggregating
#     # Note: here I'm just calculating the sum of columns C and D, but you can use any aggregation function
#     new_df = df.groupby(list(groupby_cols_subset)).agg(agg_dict).reset_index()

#     test_lst = list(set(lst_df) - set(list(new_df.columns)))

#     # # Add a new column to indicate which groupby columns are selected
#     new_df['groupby_cols'] = ', '.join(groupby_cols_subset)

#     new_col_val = []
#     for i in range(len(test_lst)):
#         # Create a new column with the appropriate number of "Z"s and the prefix
#         new_col_val.append(prefix * (i+1))

#     for i in range(len(new_col_val)):
#         # Add the new column to the DataFrame
#         new_df[test_lst[i]] = new_col_val[i]

#     i += 1
    
#     print(new_df, '\n')


In [29]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'AGE_GR',
       'STRMID', 'POP', 'LF', 'WRK'],
      dtype='object')

In [30]:
df['SEC'].unique()

array([1, 2])

**STEP 1**

In [31]:
# # Read the data from the CSV file
# df = pd.read_csv('Work_File1A.csv')

# Get the current column names
old_col_names = df.columns.tolist()

# Create a dictionary with the new column names
new_col_names = {col: col.strip() for col in old_col_names}

# Rename the columns using the dictionary
df.rename(columns=new_col_names, inplace=True)

# Calculate the SS_MULT and CMULT fields
df['SS_MULT'] = df['MULT'] / 100
df['CMULT'] = df.apply(lambda row: row['MULT'] / 200 if row['NSC'] > row['NSS'] else row['MULT'] / 100, axis=1)

lst_df = list(df.columns)

groupby_cols = GROUP_(data, df)

# Create a list of all possible subsets of groupby_cols
groupby_cols_subsets = [list(subset) for i in range(len(groupby_cols) + 1)
                        for subset in itertools.combinations(groupby_cols, i)]

# Select columns that are not in groupby_cols
cols = [col for col in df.columns if col in groupby_cols]

agg_dict = {col: sum for col in cols if (df[col].dtype == int or df[col].dtype == float) and col in groupby_cols}

################
Arr_W1 = []


### Loop over each subset of columns and perform the aggregation
### ------------------------------------------------------------
for groupby_cols_subset in groupby_cols_subsets:
    # Define the prefix to use for the new columns
    prefix = 'Z'

    if len(groupby_cols_subset) == 0:

        # # If empty list is selected, perform aggregation without grouping
        # w1 = df.agg(agg_dict).to_frame().T

        # print(new_df)

        test_lst = groupby_cols

        df['POP'] = pd.to_numeric(df['POP']) * df['CMULT']
        df['LF'] = pd.to_numeric(df['LF']) * df['CMULT']
        df['WRK'] = pd.to_numeric(df['WRK']) * df['CMULT']
        df['MULT'] = df.shape[0]

        agg_dict = {'POP': 'sum', 'LF': 'sum', 'WRK': 'sum'}

        # Group the data by SEC and ST_GR and calculate the aggregates
        w1 = df.agg(agg_dict).to_frame().T

        # Rename the columns and reset the index
        w1 = w1.rename(columns={'POP': 'pophat', 'LF': 'lfhat', 'WRK': 'wrkhat', 'MULT': 'count'}).reset_index()
        w1['groupby_cols'] = 'All columns'

        new_col_val_ = []

        for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
            # Create a new column with the appropriate number of "Z"s and the prefix
            new_col_val_.append(prefix * (i+1))

        for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
            # Add the new column to the DataFrame
            w1[test_lst[i]] = new_col_val_[i]

        # print(w1)
        ################
        Arr_W1.append(w1)

        continue

    # Create a new dataframe by grouping by the selected columns and aggregating
    # Note: here I'm just calculating the sum of columns C and D, but you can use any aggregation function
    # new_df = df.groupby(list(groupby_cols_subset)).agg(agg_dict).reset_index()
    
    # Group the data by SEC and ST_GR and calculate the aggregates
    w1 = df.groupby(groupby_cols_subset).agg({'POP': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                'LF': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                    'WRK': lambda x: (pd.to_numeric(x) * df['CMULT']).sum(),
                                                        'MULT': 'count'}).reset_index()

    # Rename the columns and reset the index
    w1 = w1.rename(columns={'POP': 'pophat', 'LF': 'lfhat', 'WRK': 'wrkhat', 'MULT': 'no_sam'}).reset_index()

    test_lst = groupby_cols_subset

    # Add a new column to indicate which groupby columns are selected
    w1['groupby_cols'] = ', '.join(groupby_cols_subset)

    new_col_val = []
    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
        # Create a new column with the appropriate number of "Z"s and the prefix
        new_col_val.append(prefix * (i+1))

    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w1.columns)))):
        # Add the new column to the DataFrame
        w1[list(set(groupby_cols_subsets[-1]) - set(w1.columns))[i]] = new_col_val[i]

    Arr_W1.append(w1)


In [32]:
Arr_W1

[   index        pophat         lfhat        wrkhat groupby_cols SEC AGE_GR
 0      0  1.088092e+09  3.844977e+08  3.556216e+08  All columns   Z     ZZ,
    index  SEC        pophat         lfhat        wrkhat  no_sam groupby_cols  \
 0      0    1  2.824017e+13  9.614018e+12  8.900422e+12   62248          SEC   
 1      1    2  4.937165e+12  1.698489e+12  1.564103e+12   47608          SEC   
 
   AGE_GR  
 0      Z  
 1      Z  ,
    index  AGE_GR        pophat         lfhat        wrkhat  no_sam  \
 0      0       0  2.888982e+12  8.854129e+11  8.625883e+11   10293   
 1      1       1  9.927197e+12  9.111205e+10  4.943696e+10   26055   
 2      2       2  3.484166e+12  6.735014e+11  4.864128e+11   12104   
 3      3       3  5.268219e+12  2.553399e+12  2.155398e+12   20108   
 4      4       5  1.160877e+13  7.109082e+12  6.910689e+12   41296   
 
   groupby_cols SEC  
 0       AGE_GR   Z  
 1       AGE_GR   Z  
 2       AGE_GR   Z  
 3       AGE_GR   Z  
 4       AGE_GR   Z  ,
    

In [33]:
df['SS'].unique()

array([1, 2])

In [34]:
df['no_sam'] = 0

In [35]:
df.columns

Index(['SEC', 'ST', 'FSU', 'IDNO', 'SEG', 'SSS', 'SSU', 'STRM', 'SSTRM', 'SS',
       'NSS', 'NSC', 'MULT', 'HH_SIZE', 'HHTYPE', 'RELG', 'SG', 'MHCE',
       'DC_ALL', 'DC_ST', 'SRL', 'REL', 'SEX', 'MSEX', 'AGE', 'MARST',
       'GEDU_LVL', 'PAS', 'PS_SS', 'CWS', 'ERN_SELF', 'ERN_REG', 'AGE_GR',
       'STRMID', 'POP', 'LF', 'WRK', 'SS_MULT', 'CMULT', 'no_sam'],
      dtype='object')

**STEP 2** IT is running slow, figure it out later...


In [36]:
# Filter dataframe to include only rows where SS is 1 or 2
df = df.query("SS in [1, 2]")
# print(f"Number of rows after filtering: {len(df)}")

Arr_W2 = []

### Loop over subset of columns to perform aggregation
for groupby_cols_subset in groupby_cols_subsets:

    new_groupby_cols_subset = ['STRMID']

    # Define the prefix to use for the new columns
    prefix = 'Z'

    if len(groupby_cols_subset) == 0:

        df['popstr'] = pd.to_numeric(df['POP']) * df['CMULT']
        df['s1pop'] = np.where(df['SS'] == 1, pd.to_numeric(df['POP']) * df['SS_MULT'], 0)
        df['s2pop'] = np.where(df['SS'] == 2, pd.to_numeric(df['POP']) * df['SS_MULT'], 0)
        df['lfstr'] = pd.to_numeric(df['LF']) * df['CMULT']
        df['s1lf'] = np.where(df['SS'] == 1, pd.to_numeric(df['LF']) * df['SS_MULT'], 0)
        df['s2lf'] = np.where(df['SS'] == 2, pd.to_numeric(df['LF']) * df['SS_MULT'], 0)
        df['wrkstr'] = pd.to_numeric(df['WRK']) * df['CMULT']
        df['s1wrk'] = np.where(df['SS'] == 1, pd.to_numeric(df['WRK']) * df['SS_MULT'], 0)
        df['s2wrk'] = np.where(df['SS'] == 2, pd.to_numeric(df['WRK']) * df['SS_MULT'], 0)

        agg_dict = {'popstr': 'sum', 's1pop': 'sum', 's2pop': 'sum', 'lfstr': 'sum', 's1lf': 'sum', \
                    's2lf': 'sum', 'wrkstr': 'sum', 's1wrk': 'sum', 's2wrk': 'sum'}

        # # If empty list is selected, perform aggregation without grouping
        w2 = df.agg(agg_dict).to_frame().T

        w2['STRMID'] = 'ALL'

        new_col_val_ = []

        for i in range(len(list(set(groupby_cols) - set(w2.columns)))):
            # Create a new column with the appropriate number of "Z"s and the prefix
            new_col_val_.append(prefix * (i+1))

        for i in range(len(list(set(groupby_cols) - set(w2.columns)))):
            # Add the new column to the DataFrame
            w2[groupby_cols[i]] = new_col_val_[i]

        print(w2)
        Arr_W2.append(w2)

        continue

    new_groupby_cols_subset.append(groupby_cols_subset[0])

    print(new_groupby_cols_subset)

    w2 = df.groupby(new_groupby_cols_subset).agg(
    popstr=('POP', lambda x: np.sum(x * df['CMULT'])),
    s1pop=('POP', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
    s2pop=('POP', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0))),
    lfstr=('LF', lambda x: np.nansum(x * df['CMULT'])),
    s1lf=('LF', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
    s2lf=('LF', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0))),
    wrkstr=('WRK', lambda x: np.nansum(x * df['CMULT'])),
    s1wrk=('WRK', lambda x: np.nansum(np.where(df['SS'] == 1, x * df['SS_MULT'], 0))),
    s2wrk=('WRK', lambda x: np.nansum(np.where(df['SS'] == 2, x * df['SS_MULT'], 0)))
    ).reset_index()

    new_col_val = []
    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w2.columns)))):
        # Create a new column with the appropriate number of "Z"s and the prefix
        new_col_val.append(prefix * (i+1))

    for i in range(len(list(set(groupby_cols_subsets[-1]) - set(w2.columns)))):
        # Add the new column to the DataFrame
        w2[list(set(groupby_cols_subsets[-1]) - set(w2.columns))[i]] = new_col_val[i]

    print(w2)
    Arr_W2.append(w2)

         popstr         s1pop         s2pop         lfstr          s1lf  \
0  3.317734e+13  3.267186e+13  3.353884e+13  1.131251e+13  1.123293e+13   

           s2lf        wrkstr         s1wrk         s2wrk STRMID SEC AGE_GR  
0  1.131725e+13  1.046452e+13  1.044359e+13  1.041990e+13    ALL   Z     ZZ  
['STRMID', 'SEC']
       STRMID  SEC        popstr         s1pop         s2pop         lfstr  \
0    01010101    1  5.545249e+11  5.147484e+11  5.943015e+11  2.044521e+11   
1    01010102    1  5.007986e+11  5.869899e+11  4.146074e+11  1.870123e+11   
2    01010103    1  4.688394e+11  4.804593e+11  4.572195e+11  1.517350e+11   
3    01010104    1  5.268955e+11  5.489126e+11  5.048784e+11  1.826084e+11   
4    01010105    1  5.228403e+11  6.134968e+11  4.321839e+11  1.673111e+11   
..        ...  ...           ...           ...           ...           ...   
236  02120304    2  4.805586e+10  9.611172e+10  0.000000e+00  2.365066e+10   
237  02130304    2  9.104503e+10  0.000000e+00  1.8

In [37]:
Arr_W3 = []

### Loop over subset of columns to perform aggregation
for i in range(len(Arr_W2)):

    # Join w2 and w1 on SEC and ST_GR columns
    # w3 = Arr_W2[i].merge(Arr_W1[i], on=['SEC', 'AGE_GR'])
    w3 = pd.concat([Arr_W2[i], Arr_W1[i]], axis=1)

    w3['no_sam'] = w3.shape[0]
    
    # Add new columns to w3
    w3['r1hat'] = w3['lfhat'] / w3['pophat']
    w3['r2hat'] = w3['wrkhat'] / w3['pophat']

    # Equate s1pop and s2pop
    w3.loc[w3['popstr'] == w3['s1pop'], 's1pop'] = w3.loc[w3['popstr'] == w3['s1pop'], 's2pop']
    w3.loc[w3['popstr'] == w3['s2pop'], 's2pop'] = w3.loc[w3['popstr'] == w3['s2pop'], 's1pop']

    # Equate s1lf and s2lf
    w3.loc[w3['lfstr'] == w3['s1lf'], 's1lf'] = w3.loc[w3['lfstr'] == w3['s1lf'], 's2lf']
    w3.loc[w3['lfstr'] == w3['s2lf'], 's2lf'] = w3.loc[w3['lfstr'] == w3['s2lf'], 's1lf']

    # Equate s1wrk and s2wrk
    w3.loc[w3['wrkstr'] == w3['s1wrk'], 's1wrk'] = w3.loc[w3['wrkstr'] == w3['s1wrk'], 's2wrk']
    w3.loc[w3['wrkstr'] == w3['s2wrk'], 's2wrk'] = w3.loc[w3['wrkstr'] == w3['s2wrk'], 's1wrk']
    
    Arr_W3.append(w3)

Arr_W3[0]

,popstr,s1pop,s2pop,lfstr,s1lf,s2lf,wrkstr,s1wrk,s2wrk,STRMID,...,index,pophat,lfhat,wrkhat,groupby_cols,SEC,AGE_GR,no_sam,r1hat,r2hat
0,3.317734e+13,3.267186e+13,3.353884e+13,1.131251e+13,1.123293e+13,1.131725e+13,1.046452e+13,1.044359e+13,1.041990e+13,ALL,...,0,1.088092e+09,3.844977e+08,3.556216e+08,All columns,Z,ZZ,1,0.353369,0.32683


In [38]:
w3.columns

Index(['STRMID', 'SEC', 'popstr', 's1pop', 's2pop', 'lfstr', 's1lf', 's2lf',
       'wrkstr', 's1wrk', 's2wrk', 'AGE_GR', 'index', 'SEC', 'AGE_GR',
       'pophat', 'lfhat', 'wrkhat', 'no_sam', 'groupby_cols', 'r1hat',
       'r2hat'],
      dtype='object')

In [39]:
Arr_W4 = []

### Loop over subset of columns to perform aggregation
for i in range(len(Arr_W3)):
    mask = Arr_W3[i].columns.duplicated()
    Arr_W3[i] = Arr_W3[i].loc[:, ~mask]

    gby_list = ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat'] + groupby_cols

    print(gby_list)

    w4 = Arr_W3[i].groupby(gby_list).agg(
        var_pop = ('s1pop', lambda x: np.sum((x - Arr_W3[i]['s2pop']) ** 2)),

        mse_R1 = ('s1lf', lambda x: np.nansum(
                    ((x - Arr_W3[i]['s2lf']) ** 2) +
                        ((Arr_W3[i]['r1hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                            (2 * Arr_W3[i]['r1hat'] * (x - Arr_W3[i]['s2lf']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                )),

        mse_R2 = ('s1wrk', lambda x: np.nansum(
                    ((x - Arr_W3[i]['s2wrk']) ** 2) +
                        ((Arr_W3[i]['r2hat'] ** 2) * ((Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop']) ** 2) -
                            (2 * Arr_W3[i]['r2hat'] * (x - Arr_W3[i]['s2wrk']) * (Arr_W3[i]['s1pop'] - Arr_W3[i]['s2pop'])))
                ),
        
        
    )).reset_index()

    Arr_W4.append(w4)

    print(w4)

['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'AGE_GR']
   no_sam        pophat         lfhat        wrkhat     r1hat    r2hat SEC  \
0       1  1.088092e+09  3.844977e+08  3.556216e+08  0.353369  0.32683   Z   

  AGE_GR       var_pop        mse_R1        mse_R2  
0     ZZ  7.516557e+23  4.930147e+22  9.428055e+22  
['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'AGE_GR']
   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat  SEC  \
0     241  4.937165e+12  1.698489e+12  1.564103e+12  0.344021  0.316802    1   
1     241  2.824017e+13  9.614018e+12  8.900422e+12  0.340438  0.315169    1   

  AGE_GR       var_pop        mse_R1        mse_R2  
0      Z  2.971574e+22  7.455427e+20  1.053570e+21  
1      Z  6.328698e+21  2.700248e+20  2.079021e+18  
['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'SEC', 'AGE_GR']
   no_sam        pophat         lfhat        wrkhat     r1hat     r2hat SEC  \
0    1202  2.888982e+12  8.854129

In [40]:
Arr_W5 = []
for i in range(len(Arr_W4)):
    # Calculate RSEs for each variable in T1
    w5 = Arr_W4[i].assign(
        RSE_POP=lambda x: 100 * (x['var_pop'] ** 0.5) / (2 * x['pophat']),
        RSE_R1=lambda x: 100 * (x['mse_R1'] ** 0.5) / (2 * x['lfhat']),
        RSE_R2=lambda x: 100 * (x['mse_R2'] ** 0.5) / (2 * x['wrkhat'])
    )

    Arr_W5.append(w5)

In [41]:
Arr_W5

[   no_sam        pophat         lfhat        wrkhat     r1hat    r2hat SEC  \
 0       1  1.088092e+09  3.844977e+08  3.556216e+08  0.353369  0.32683   Z   
 
   AGE_GR       var_pop        mse_R1        mse_R2       RSE_POP  \
 0     ZZ  7.516557e+23  4.930147e+22  9.428055e+22  39839.491798   
 
          RSE_R1        RSE_R2  
 0  28873.948334  43171.078438  ,
    no_sam        pophat         lfhat        wrkhat     r1hat     r2hat  SEC  \
 0     241  4.937165e+12  1.698489e+12  1.564103e+12  0.344021  0.316802    1   
 1     241  2.824017e+13  9.614018e+12  8.900422e+12  0.340438  0.315169    1   
 
   AGE_GR       var_pop        mse_R1        mse_R2   RSE_POP    RSE_R1  \
 0      Z  2.971574e+22  7.455427e+20  1.053570e+21  1.745764  0.803792   
 1      Z  6.328698e+21  2.700248e+20  2.079021e+18  0.140851  0.085461   
 
      RSE_R2  
 0  1.037616  
 1  0.008100  ,
    no_sam        pophat         lfhat        wrkhat     r1hat     r2hat SEC  \
 0    1202  2.888982e+12  8.854129e

In [42]:
Tables = []

for i in range(len(Arr_W5)):
    gby_list = ['no_sam', 'pophat', 'lfhat', 'wrkhat', 'r1hat', 'r2hat', 'RSE_POP', 'RSE_R1', 'RSE_R2'] + groupby_cols
    # assuming the data is loaded into a DataFrame called w5
    T1 = Arr_W5[i][gby_list].copy()

    # add empty columns for sec_desc and st_gr_desc
    T1['sec_desc'] = ''
    T1['st_gr_desc'] = ''

    # apply formatting to columns
    T1['R1'] = 100 * T1['r1hat']
    T1['R2'] = 100 * T1['r2hat']

    gby_list = ['sec_desc', 'st_gr_desc', 'no_sam', 'pophat','lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2'] + groupby_cols

    # reorder columns
    T1 = T1[gby_list]

    gby_list = ['no_sam', 'pophat','lfhat', 'wrkhat', 'r1hat', 'r2hat', 'RSE_POP', 'RSE_R1', 'RSE_R2'] + groupby_cols

    T1 = Arr_W5[i][gby_list].copy()

    T1['sec_desc'] = ' '
    T1['st_gr_desc'] = ' '

    T1['R1'] = 100 * T1['r1hat']
    T1['R2'] = 100 * T1['r2hat']

    gby_list = ['no_sam', 'pophat','lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2', 'RSE_R2'] + groupby_cols

    T1 = T1[gby_list]

    Tables.append(T1)
    
print(Tables)

[   no_sam        pophat         lfhat        wrkhat       RSE_POP        R1  \
0       1  1.088092e+09  3.844977e+08  3.556216e+08  39839.491798  35.33687   

         RSE_R1         R2        RSE_R2 SEC AGE_GR  
0  28873.948334  32.683043  43171.078438   Z     ZZ  ,    no_sam        pophat         lfhat        wrkhat   RSE_POP         R1  \
0     241  4.937165e+12  1.698489e+12  1.564103e+12  1.745764  34.402113   
1     241  2.824017e+13  9.614018e+12  8.900422e+12  0.140851  34.043767   

     RSE_R1         R2    RSE_R2  SEC AGE_GR  
0  0.803792  31.680175  1.037616    1      Z  
1  0.085461  31.516886  0.008100    1      Z  ,    no_sam        pophat         lfhat        wrkhat   RSE_POP         R1  \
0    1202  2.888982e+12  8.854129e+11  8.625883e+11  0.095322  30.647922   
1    1202  3.484166e+12  6.735014e+11  4.864128e+11  0.014552  19.330349   
2    1202  5.268219e+12  2.553399e+12  2.155398e+12  0.031846  48.467969   
3    1202  9.927197e+12  9.111205e+10  4.943696e+10  0.3

In [43]:
def merge_dataframes(tables):
    merged_df = tables[0]
    for df in tables[1:]:
        merged_df = pd.concat([merged_df, df], axis=0)
    return merged_df

merge_dataframes(Tables)

,no_sam,pophat,lfhat,wrkhat,RSE_POP,R1,RSE_R1,R2,RSE_R2,SEC,AGE_GR
0,1,1.088092e+09,3.844977e+08,3.556216e+08,39839.491798,35.336870,28873.948334,32.683043,43171.078438,Z,ZZ
0,241,4.937165e+12,1.698489e+12,1.564103e+12,1.745764,34.402113,0.803792,31.680175,1.037616,1,Z
1,241,2.824017e+13,9.614018e+12,8.900422e+12,0.140851,34.043767,0.085461,31.516886,0.008100,1,Z
0,1202,2.888982e+12,8.854129e+11,8.625883e+11,0.095322,30.647922,0.058649,29.857865,0.057679,Z,0
1,1202,3.484166e+12,6.735014e+11,4.864128e+11,0.014552,19.330349,0.608802,13.960667,0.804055,Z,2
2,1202,5.268219e+12,2.553399e+12,2.155398e+12,0.031846,48.467969,0.101411,40.913207,0.251893,Z,3
3,1202,9.927197e+12,9.111205e+10,4.943696e+10,0.316966,0.917802,0.316966,0.497995,0.316966,Z,1
4,1202,1.160877e+13,7.109082e+12,6.910689e+12,0.076492,61.238879,0.084266,59.529891,0.087117,Z,5
0,241,4.017032e+11,7.656468e+10,7.425642e+10,18.227746,19.060012,40.683005,18.485393,46.626581,1,Z
1,241,4.633418e+11,9.749668e+10,7.866387e+10,0.306893,21.042062,7.304186,16.977503,19.545224,1,Z


In [44]:
# T1.to_csv('T1_Balanced_Half_Sample_Method.csv')

In [45]:
T1.columns

Index(['no_sam', 'pophat', 'lfhat', 'wrkhat', 'RSE_POP', 'R1', 'RSE_R1', 'R2',
       'RSE_R2', 'SEC', 'AGE_GR'],
      dtype='object')